In [ ]:
# load videos
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import time
import pickle

import npc_lims
from npc_sessions import DynamicRoutingSession, get_video_data

In [ ]:
DR_ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True) 
                          if s.is_uploaded and s.is_annotated and
                          s.project=='DynamicRouting')
# Templeton_ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True)
#                                 if s.is_uploaded and s.is_annotated and
#                                 s.project=='TempletonPilotSession')

In [ ]:
DR_ephys_sessions[18]

In [ ]:
# save_path = r'\\allen\programs\mindscope\workgroups\templeton\TTOC\average video frames and motion'
save_path = r'\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\average video frames and motion'

In [ ]:
session.trials[:].columns

In [ ]:
# session.is_task
sel_session

In [ ]:
#set amount of time to average over
prestim_time=0.2
except_list={}

for sel_session in range(0,18):
    try:
        #loop through selected sessions
        # sel_session=0
        session_info=DR_ephys_sessions[sel_session]
        # session_info=Templeton_ephys_sessions[sel_session]

        print('loading session: '+session_info.id)

        session=DynamicRoutingSession(session_info.allen_path)

        #check if video exists
        if session.is_video==False:
            print('no video for session ',session_info.id,'; skipping')
            continue

        if session.is_task == False:
            print('no task for session ',session_info.id,'; skipping')
            continue

        vid_angles=['behavior','face']
        vid_angle_npc_names={
            'behavior':'side',
            'face':'front',
            'eye':'eye',
            }

        #load camera frames
        cam_frames={}
        for vid_angle in vid_angles:
            for frame_time in session._video_frame_times:
                if vid_angle_npc_names[vid_angle] in frame_time.name:
                    cam_frames[vid_angle]=frame_time.timestamps
                    break

                
        #load video
        vid_paths={}
        for vid_angle in vid_angles:
            for vid_path in session.video_paths:
                if vid_angle in str(vid_path).lower() and 'zip' not in str(vid_path):
                    vid_paths[vid_angle]=vid_path
                    break


        for vid_angle in vid_angles:
            cap = get_video_data(vid_paths[vid_angle])
            framecount = 0
            trialcount = 0
            trialframecount = 0

            fps, frames = cap.get(cv2.CAP_PROP_FPS), cap.get(cv2.CAP_PROP_FRAME_COUNT)
            w_frame, h_frame = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            n_prestim_frames=int(fps*prestim_time)


            #find frame times for vis vs. aud blocks
            prestim_frames=[]

            ####NEED TO USE THESE TO FIND THE RIGHT FRAMES!!

            trials=pd.read_parquet(
                    npc_lims.get_cache_path('trials',session.id,version='any')
                )


            for tt,trial in trials[:].iterrows():
                if len(np.where(cam_frames[vid_angle]>=trial['stim_start_time'])[0])>0:
                    stimStartFrame=np.where(cam_frames[vid_angle]>=trial['stim_start_time'])[0][0]
                else:
                    continue

                trial_prestim_frames = np.arange(
                    stimStartFrame-n_prestim_frames,
                    stimStartFrame
                )

                prestim_frames.append(trial_prestim_frames)
            
            if len(prestim_frames)>0:
                prestim_frames=np.vstack(prestim_frames)
            else:
                continue

            #pre-allocate arrays: frames and frame diffs
            avg_prestim_frames=np.zeros((h_frame,w_frame,prestim_frames.shape[0]),dtype=int)
            avg_prestim_motion=np.zeros((h_frame,w_frame,prestim_frames.shape[0]),dtype=int)

            temp_frames=np.zeros((h_frame,w_frame,n_prestim_frames),dtype=int)

            #loop through frames, save frames we want to arrays + save diffs btwn frames (avg per trial)
            lastpct=0
            start_time=time.time()
            while(cap.isOpened()):
                ret, frame = cap.read()

                # average arrays
                if ret==True:

                    # Percentage of frames completed
                    xx = framecount *100/frames
                    xx=int(xx)
                    if (xx%10==0)&(xx>lastpct):
                        lastpct=xx
                        elapsed_time=time.time()-start_time
                        print(int(xx),'% complete; time elapsed:',np.round(elapsed_time,decimals=2))

                    if trialcount < prestim_frames.shape[0]:
                        if framecount in prestim_frames[trialcount,:]:
                            temp_frames[:,:,trialframecount]=np.mean(frame,axis=2)
                            trialframecount+=1
                            if trialframecount>n_prestim_frames-1:
                                temp_avg_frame = np.mean(temp_frames,axis=2)
                                temp_avg_mot = np.mean(np.abs(np.diff(temp_frames,axis=2)),axis=2)

                                avg_prestim_frames[:,:,trialcount]=temp_avg_frame
                                avg_prestim_motion[:,:,trialcount]=temp_avg_mot

                                trialcount+=1
                                trialframecount=0
                                    
                                temp_frames=np.zeros((h_frame,w_frame,n_prestim_frames),dtype=int)  

                else:

                    break

                framecount += 1

            elapsed_time=time.time()-start_time
            print('100% complete; time elapsed:',np.round(elapsed_time,decimals=2))    
            cap.release()

            #save data to disk
            file_save_path=os.path.join(save_path,vid_angle,session.id+'_trial_avg_frames.npz')
            np.savez(file_save_path,avg_prestim_frames=avg_prestim_frames,
                    avg_prestim_motion=avg_prestim_motion,prestim_time=prestim_time,
                    trials=session.trials[:],session_id=session.id)
            
    except Exception as e:
        except_list[session.id]=repr(e)



In [ ]:
file_save_path=os.path.join(save_path,vid_angle,session.id+'_trial_avg_frames.npz')
np.savez(file_save_path,avg_prestim_frames=avg_prestim_frames,
         avg_prestim_motion=avg_prestim_motion,prestim_time=prestim_time,
         trials=session.trials[:],session_id=session.id)

In [ ]:
session.id


In [ ]:
sel_session

In [ ]:
except_list